## Model

In [ ]:
!pip install flair

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 88.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 78.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 68.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 59.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 KB 58.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import flair, torch
from flair.models import SequenceTagger
from flair.tokenization import SegtokSentenceSplitter
from flair.data import Sentence

from collections import defaultdict

In [ ]:
torch.cuda.is_available()

In [ ]:
!nvidia-smi

In [ ]:
flair.device = torch.device("cuda:0")

In [ ]:
splitter = SegtokSentenceSplitter()
tagger = SequenceTagger.load('ner')
# tagger = SequenceTagger.load('ner-fast')

In [ ]:
def ner_eval(stats: dict, new: dict):
    splitted = splitter.split(new["content"])
    splitted.append(Sentence(new["title"]))
    if isinstance(new["description"], str):
        splitted.extend(
            splitter.split(new["description"])
        )

    tagger.predict(splitted)

    buf_stats = {
        "PER":  defaultdict(int),
        "LOC":  defaultdict(int),
        "ORG":  defaultdict(int),
        "MISC": defaultdict(int)
    }     # костыль

    # число вхождений в текст
    for sentence in splitted:
        for entity in sentence.get_spans("ner"):
            #print(entity)
            buf_stats[entity.tag][entity.text] += 1 

    for key in buf_stats.keys():
        for entity in buf_stats[key].keys():
            stats[key][entity].append((new["index"], buf_stats[key][entity]))

## Data

In [ ]:
from pprint import pprint
import json
from google.colab import drive

In [ ]:
drive.mount('/content/up_nlp/', force_remount=True)

In [ ]:
with open("/content/up_nlp/MyDrive/up_nlp/data.json", "r") as data_file:
    data = json.load(data_file)

len(data)

42635

In [ ]:
with open("/content/up_nlp/MyDrive/up_nlp/stats.json", "r") as stats_file:
    stats = json.load(stats_file)

for key in ["PER", "LOC", "ORG", "MISC"]:
    stats[key] = defaultdict(list, stats[key])

{key: len(stats[key]) for key in ["PER", "LOC", "ORG", "MISC"]}

{'PER': 27780, 'LOC': 7065, 'ORG': 18536, 'MISC': 11428}

## Evaluating

In [ ]:
from tqdm import tqdm

In [ ]:
N_SAMPLES = 2635

In [ ]:
len(data)

42635

In [ ]:
stats["last_new_indx"]

39999

In [ ]:
ner_samples_failed = []
for i in tqdm(range(stats["last_new_indx"]+1, stats["last_new_indx"]+N_SAMPLES+1)):
    try:
        ner_eval(stats, data[i])
    except:
        ner_samples_failed.append(data[i]["index"])

    if (i+1) % 1000 == 0:
        stats["last_new_indx"] += 1000
        with open("/content/up_nlp/MyDrive/up_nlp/stats.json", "w") as stats_file:
            stats_file.write(json.dumps(stats))
            print(f"Dumped after {(stats['last_new_indx'])}'th sample")
print(ner_samples_failed)

 38%|███▊      | 1000/2635 [07:10<30:27,  1.12s/it]

Dumped after 40999'th sample


 76%|███████▌  | 2000/2635 [13:44<12:55,  1.22s/it]

Dumped after 41999'th sample


100%|██████████| 2635/2635 [18:08<00:00,  2.42it/s]

[]


In [ ]:
{key: len(stats[key]) for key in ["PER", "LOC", "ORG", "MISC"]}

{'PER': 185089, 'LOC': 52875, 'ORG': 108032, 'MISC': 88084}

In [ ]:
del stats['last_new_indx']

In [ ]:
with open("/content/up_nlp/MyDrive/up_nlp/stats.json", "w") as stats_file:
    stats_file.write(json.dumps(stats))

In [ ]:
stats["last_new_indx"]

KeyError: ignored